# Biogas Production Calculations

## Determine Biogas Production Based on Design Parameters 

### Biogas Production Calculations

**Critical Assumptions:**
1. WW_gen: rule of thumb for wastewater production per person, from Monroe
2. HRT: Number taken from previous lab scale tests and literature reviews (but may still be low)
3. COD_rem: Assumed efficency of reactor for breaking down COD, general number from the literature
4. BG_Max: Biogas production calculated from theoretical efficency, using stoichiometry of reaction.  Detailed in Spring 2014 report
5. BG_Min: Minimum biogas production from COD load, taken from lower bound in [Van Lier report](https://courses.edx.org/c4x/DelftX/CTB3365STx/asset/Chap_4_Van_Lier_et_al.pdf)
6. BG_Avg: Middle value, taken from average output in Aguaclara reactors (in Spring 2014 report)
7. SH_high: Assuming maximum efficency of a stove, using results from [SSWM](https://www.sswm.info/content/direct-use-biogas)
8. SH_low: Assuming minimum efficency of a stove, using results from [SSWM](https://www.sswm.info/content/direct-use-biogas)

In [6]:
from aide_design.play import*
import math

#Calculate Dimensions and Flow Rate 
def UASBSize(Diam, Height):
    WW_gen = 0.003 * u.L/u.s #Wastewater generated per person
    HRT = 4 * u.hr #Hydraulic Residence Time
    Vol = math.pi * (Diam / 2)**2 * Height / 2
    Flow = Vol.to(u.L) / HRT.to(u.s)
    PS = Flow / WW_gen #People served per reactor
    return (Vol, Flow)

#Calculate Biogas Rate of Production (L/s) in UASB
def BiogasProd(Flow, COD_Load):
    COD = Flow * COD_Load
    COD_rem = COD * 0.7 #Assuming x% efficency of COD removal and conversion in reactor
    BGMax = COD_rem * 0.378 *(u.ml/u.mg) #Theoretical Productiom, from Fall 2014 UASB team report
    BGMin = COD_rem * 0.06 *(u.ml/u.mg) #Production using minimum value from Van Lier report
    BGAvg = COD_rem * 0.16 *(u.ml/u.mg) #Production using average value from Spring 2014 tests
    return (BGMax, BGAvg, BGMin)

#Calculate Energy Production from Biogas Produced
def EnergyProduction(Biogas):
    Biogas = Biogas.to(u.L/u.days)
    SH_high = Biogas / (200 * u.L) #Hours of stove usage, given maximum efficency of stove
    SH_low = Biogas / (450 * u.L) #Hours of stove usage, given minimum efficency
    KWH = Biogas / (700 * u.L/u.kwh) #Kilowatt Hours generated from biogas used 
    return


#Biogas Production from Influent COD (estimates)

Headspace Design Functions